<a href="https://colab.research.google.com/github/eamonbyrne/protein_ML/blob/main/protein_gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying to implement a mini-protein generator (a la "infinite Shakespeare") based upon Karpathy's PyTorch decoder.

Based upon the Andrej Karpathy gpt-dev notebook (https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing) and YouTube video "Let's build GPT: from scratch, in code, spelled out." (https://www.youtube.com/watch?v=kCc8FmEb1nY).

Uses a Causal Language Model (i.e. the model is trained to predict the next token in the sequence, given the previous tokens).

In [ ]:
# Start with a dataset to train on.

# Use the "train_sequences.fasta" file from the CAFA5 Kaggle competition
# (https://www.kaggle.com/competitions/cafa-5-protein-function-prediction/overview).

# This is a text file that contains the amino acid sequences of 142264 proteins
# in fasta format (i.e. each uppercase letter represents an amino acid, 60 letters
# per line, each protein is a block of amino acids, each protein-block begins with
# a ">" symbol followed by a plain text title on a single line, the protein sequence
# begins on the line after the ">" symbol).

In [ ]:
# read text (fasta) file in to inspect it
with open('/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences.fasta', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  97263064


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

>P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1
MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLKQPLRNK
RVCVCGIDPYPKDGTGVPFESPNFTKKSIKEIASSISRLTGVIDYKGYNLNIIDGVIPWN
YYLSCKLGETKSHAIYWDKISKLLLQHITKHVSVLYCLGKTDFSNIRAKLESPVTTIVGY
HPAARDRQFEKDRSFEIINVLLELDNKVPINWAQGFIY
>O73864 sp|O73864|WNT11_DANRE Protein Wnt-11 OS=Danio rerio OX=7955 GN=wnt11 PE=2 SV=1
MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKLLDGLVPDQQQLCKR
NLELMHSIVRAARLTKSACTSSFSDMRWNWSSIESAPHFTPDLAKGTREAAFVVSLAAAV
VSHAIARACASGDLPSCSCAAMPSEQAAPDFRWGGCGDNLRYYGLQMGSAFSDAPMRNRR
SGPQDFRLMQLHNNAVGRQVLMDSLEMKCKCHGVSGSCSVKTCWKGLQDISTISADLKSK
YLSATKVIPRQIGTRRQLVPREMEVRPVGENELVYLVSSPDYCTQNAKQGSLGTTDRQCN
KTASGSESCGLMCCGRGYNAYTEVLVERCQCKYHWCCYVSCKTCKRTVERYVSK
>O95231 sp|O95231|VENTX_HUMAN Homeobox protein VENTX OS=Homo sapiens OX=9606 GN=VENTX PE=1 SV=1
MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLPGPGQTSGAREPPQA
VSIKEAAGSSNLPAPERTMAGLSKEPNTLRAPRVRTAFTMEQVRTLEGVFQHHQYLSPL

In [ ]:
# Want to get rid of the heading line text for each protein,
# but keep the ">" as a START token.

input_file_path = "/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences.fasta"
output_file_path = "/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences_cleaned.fasta"

with open(input_file_path, "r") as input_file, open(output_file_path, "w") as output_file:
    for line in input_file:
        if line.startswith(">"):
            output_file.write(">\n")
        else:
            output_file.write(line)


In [ ]:
# read text (fasta) file in to inspect it
with open('/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences_cleaned.fasta', 'r', encoding='utf-8') as f:
    text_cleaned = f.read()

print(text_cleaned[:1000])

>
MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLKQPLRNK
RVCVCGIDPYPKDGTGVPFESPNFTKKSIKEIASSISRLTGVIDYKGYNLNIIDGVIPWN
YYLSCKLGETKSHAIYWDKISKLLLQHITKHVSVLYCLGKTDFSNIRAKLESPVTTIVGY
HPAARDRQFEKDRSFEIINVLLELDNKVPINWAQGFIY
>
MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKLLDGLVPDQQQLCKR
NLELMHSIVRAARLTKSACTSSFSDMRWNWSSIESAPHFTPDLAKGTREAAFVVSLAAAV
VSHAIARACASGDLPSCSCAAMPSEQAAPDFRWGGCGDNLRYYGLQMGSAFSDAPMRNRR
SGPQDFRLMQLHNNAVGRQVLMDSLEMKCKCHGVSGSCSVKTCWKGLQDISTISADLKSK
YLSATKVIPRQIGTRRQLVPREMEVRPVGENELVYLVSSPDYCTQNAKQGSLGTTDRQCN
KTASGSESCGLMCCGRGYNAYTEVLVERCQCKYHWCCYVSCKTCKRTVERYVSK
>
MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLPGPGQTSGAREPPQA
VSIKEAAGSSNLPAPERTMAGLSKEPNTLRAPRVRTAFTMEQVRTLEGVFQHHQYLSPLE
RKRLAREMQLSEVQIKTWFQNRRMKHKRQMQDPQLHSPFSGSLHAPPAFYSTSSGLANGL
QLLCPWAPLSGPQALMLPPGSFWGLCQVAQEALASAGASCCGQPLASHPPTPGRPSLGPA
LSTGPRGLCAMPQTGDAF
>
MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEPVALRGLRLEAQGRA
TSAWGPSAGARVCIGGGSPAASSEVEYLNLRLSLLEAPAGEGVTLLQPGKHEFPFRFQLP
SEPLATSFTGKYGSIQYCVRAVLER

In [ ]:
print("length of dataset in characters: ", len(text_cleaned))

length of dataset in characters:  80419829


In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text_cleaned)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


>ABCDEFGHIKLMNOPQRSTUVWXYZ
27


Hmmm what are the amino acids represented by O, U, X, Z ? These are non-canonical amino acids. E.g. X = undetermined amino acid; Z = glutamine or glutamic acid.

One of the characters (it will be "0" once numericalized) in the vocab is new_line.

In [ ]:
# create a mapping from characters to integers (i.e. tokenize and numericalize)
stoi = { ch:i for i,ch in enumerate(chars) } # stoi => string to integer
itos = { i:ch for i,ch in enumerate(chars) } # itos => integer to string
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode(">MGGEAGADG"))
print(decode(encode(">MGGEAGADG")))

[1, 13, 8, 8, 6, 2, 8, 2, 5, 8]
>MGGEAGADG


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text_cleaned), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([80419829]) torch.int64
tensor([ 1,  0, 13, 14, 19, 22, 20, 22, 19,  9,  2, 16, 25, 20, 10, 20, 25,  9,
         5,  5, 23,  6, 16, 22, 13, 19, 17, 12, 22,  6,  7, 25, 14,  6, 22,  2,
        19, 23, 12, 12, 18,  5,  6, 20, 19, 16, 10, 16,  5, 11,  7,  7, 10, 17,
        12, 11, 17, 16, 12, 18, 14, 11,  0, 18, 22,  4, 22,  4,  8, 10,  5, 16,
        25, 16, 11,  5,  8, 20,  8, 22, 16,  7,  6, 19, 16, 14,  7, 20, 11, 11,
        19, 10, 11,  6, 10,  2, 19, 19, 10, 19, 18, 12, 20,  8, 22, 10,  5, 25,
        11,  8, 25, 14, 12, 14, 10, 10,  5,  8, 22, 10, 16, 23, 14,  0, 25, 25,
        12, 19,  4, 11, 12,  8,  6, 20, 11, 19,  9,  2, 10, 25, 23,  5, 11, 10,
        19, 11, 12, 12, 12, 17,  9, 10, 20, 11,  9, 22, 19, 22, 12, 25,  4, 12,
         8, 11, 20,  5,  7, 19, 14, 10, 18,  2, 11, 12,  6, 19, 16, 22, 20, 20,
        10, 22,  8, 25,  0,  9, 16,  2,  2, 18,  5, 18, 17,  7,  6, 11,  5, 18,
        19,  7,  6, 10, 10, 14, 22, 12, 12,  6, 12,  5, 14, 11, 22, 16, 10, 14,
     

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# We are going to feed the text file into the decoder with a certain amount of
# context (this will be increased in size later on). This is later referred to
# as "time, T" because each token is sequential; one must be predicted before
# the next one.

block_size = 8
train_data[:block_size+1]

tensor([ 1,  0, 13, 14, 19, 22, 20, 22, 19])

In [ ]:
# We are training the decoder to predict the next amino acid in the sequence,
# so the labels (y) will be the next amino acid after the feature/s (x).

x = train_data[:block_size]   # features
y = train_data[1:block_size+1]   # labels (offset from features by +1, i.e. the next number in the sequence)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 0
when input is tensor([1, 0]) the target: 13
when input is tensor([ 1,  0, 13]) the target: 14
when input is tensor([ 1,  0, 13, 14]) the target: 19
when input is tensor([ 1,  0, 13, 14, 19]) the target: 22
when input is tensor([ 1,  0, 13, 14, 19, 22]) the target: 20
when input is tensor([ 1,  0, 13, 14, 19, 22, 20]) the target: 22
when input is tensor([ 1,  0, 13, 14, 19, 22, 20, 22]) the target: 19


In [ ]:
block_size = 8
torch.randint(len(data)-block_size, (batch_size,))

tensor([31371159,  9700037, 63874338, 50735187])

In [ ]:
x = torch.stack([data[31371159:31371159+block_size]])
x

tensor([[22,  5, 25, 10, 10,  7,  8, 20]])

In [ ]:
# We will run multiple batches (i.e. multiple "blocks" of text) at the same
# time because we want to utilize parallel processing (enabled by GPUs).

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate a set of batch starting positions: returns a tensor filled with batch_size number of integers from within the data indices
    x = torch.stack([data[i:i+block_size] for i in ix]) # concatenates a sequence of tensors (from the data, referenced by index) along a new dimension
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # same as for x except +1
    return x, y

xb, yb = get_batch('train')
print('INPUTS:')
print(xb.shape)
print(xb)
print('TARGETS:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

INPUTS:
torch.Size([4, 8])
tensor([[16,  2,  6,  9,  8, 23, 11,  6],
        [18, 14,  6, 12,  2, 17,  9, 13],
        [ 8, 12,  0,  2, 10,  5,  9, 18],
        [12,  8, 19,  7, 10, 19,  8, 17]])
TARGETS:
torch.Size([4, 8])
tensor([[ 2,  6,  9,  8, 23, 11,  6, 19],
        [14,  6, 12,  2, 17,  9, 13, 17],
        [12,  0,  2, 10,  5,  9, 18, 11],
        [ 8, 19,  7, 10, 19,  8, 17, 12]])
----
when input is [16] the target: 2
when input is [16, 2] the target: 6
when input is [16, 2, 6] the target: 9
when input is [16, 2, 6, 9] the target: 8
when input is [16, 2, 6, 9, 8] the target: 23
when input is [16, 2, 6, 9, 8, 23] the target: 11
when input is [16, 2, 6, 9, 8, 23, 11] the target: 6
when input is [16, 2, 6, 9, 8, 23, 11, 6] the target: 19
when input is [18] the target: 14
when input is [18, 14] the target: 6
when input is [18, 14, 6] the target: 12
when input is [18, 14, 6, 12] the target: 2
when input is [18, 14, 6, 12, 2] the target: 17
when input is [18, 14, 6, 12, 2, 17] the t

In [ ]:
print(xb) # our input to the transformer

tensor([[16,  2,  6,  9,  8, 23, 11,  6],
        [18, 14,  6, 12,  2, 17,  9, 13],
        [ 8, 12,  0,  2, 10,  5,  9, 18],
        [12,  8, 19,  7, 10, 19,  8, 17]])


In [ ]:
# Create a model (Bigram Language Model).

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), B = batch size (here, 4), T = time (i.e. block size; here, 8), C = channel size (i.e. vocab_size; here, 65)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 27])
tensor(3.6539, grad_fn=<NllLossBackward0>)

W


PERQLSOPDSIBAZGYFF
LVVWGDHRSO>LVVVTTOPDLOPLQIRHMTZBHTVTFDPNPGU
P>S
VUZGXWGKLZIRNKZBVTF>VLEPNH>RN


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
# train the model by passing batches into it.
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.939000129699707


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


YVRITWFKKDVSKPEFRLSEYNIMSCFDFTGYDPLARTKQDSLYGKKPIQPGGSFSQEPVKACVQPKSYSNYQRQECDY
GQEIGYRSETSG
MFSKKQLPSQAVADATFSTSQSALKVCDAASCMDVGVYRLAEPILAGVPELTLHYVVGTISEGTCRAYALNNVKLHRLAKLAAEVKYKTASSWAGGQINSLFIPPQLFQSLRLTCEIFVWNANRLMLLNGNFEYFVCSLQEKAGDQATTFACYSAAGHREYYLTPDLVHELDIMVGAGVI
LLRNEGYNRRRASIMKEELQPCFNLKHSDLLN
KIKFDTHLIICEKAIGDELLPYST
ERWHKLTTCYLTLL
QSSQEVDLKEVYSYYFHKSGKFSKGS
YELYRKDGNPSKVKITAGIAATHPRALLPKCNYQVSQIRRQQPQPVAITVSRTQTTERKRQTSGAKMSSPLHNSEERRVDQGKFHWYLSQKHEVIGISVSHRGSENPS
NIAIPSTIQFIFV
>
K


In [ ]:
# Example of generating weights tensor (this is not specific to this
# protein task) for attention.

# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)   # (B, T, 16)
out = wei @ v   # (B, T, T) @ (B, T, 16) ---> ...
#out = wei @ x

out.shape
#wei.shape

torch.Size([4, 8, 16])

In [ ]:
# Let's have a look at the first weights tensor:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
# ...

class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

## Full finished code

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd =384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# take fast file input and clean it by removing title text, save as new file with "_cleaned"
input_file_path = "/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences.fasta"
output_file_path = "/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences_cleaned.fasta"
with open(input_file_path, "r") as input_file, open(output_file_path, "w") as output_file:
    for line in input_file:
        if line.startswith(">"):
            output_file.write(">\n")
        else:
            output_file.write(line)

# open cleaned file in read-mode
with open('/content/drive/MyDrive/Colab Notebooks/protein-gpt-dev/train_sequences_cleaned.fasta', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad() # tells pytorch that the following block will not require backpropagation calculations (saves memory and speeds up processes)
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)     # (B,T,C)
        q = self.query(x)   # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))   # (B, T, T)
        wei = F.softmax(wei, dim=-1)   # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)   # (B,T,C)
        out = wei @ v   # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


10.759707 M parameters
step 0: train loss 3.4883, val loss 3.4893
step 500: train loss 2.8437, val loss 2.8400
step 1000: train loss 2.8132, val loss 2.8054
step 1500: train loss 2.8064, val loss 2.8024
step 2000: train loss 2.8028, val loss 2.7961
step 2500: train loss 2.8001, val loss 2.7954
step 3000: train loss 2.7979, val loss 2.7917
step 3500: train loss 2.7944, val loss 2.7882
step 4000: train loss 2.7911, val loss 2.7850
step 4500: train loss 2.7936, val loss 2.7860
step 4999: train loss 2.7890, val loss 2.7827

NPVPPPRELLSESSFTTSSSLEDDPDSLPSSFLSVGDSESPFSLRKGSLEQAFGVLPKEF
ILDDDSSNSLPRPQDPFCDNSSSDTAFDFQPDLRWKKRTANKQGMTWVRTSMEGETERYT
EDLTGVCVMLRNWFCKKDIPRFDAAFNAQHLVTKQAVNPERDLINSLHRRPVQEIYYDVI
ATPLSMRDESCIKAKSIVMFAASATTVSVTGTDTRFLIKTIKLYKKTIDECSADAFYGLK
>
MAAQLLLSTVVVPLLFLLTFFVLIGDYGLCTSTEVVIFSALSTNGFVAASHAQFSVTYTA
GICVSIVSASLYFLFSFAVAIGYDYLEDHRLLAILLMILGMTLEGCLGYTVFMVVNPEMR
CLVLQYFPVLTLTSFGAMSDGNFLLKDGQDNNNSNNNSENENEGKPDRQQCMALLMACLG
VRGSLDFNIHTAVYTCLLNLANRFQIQAVGNIGFKTISGYAKKGH

In [ ]:
# Timing: Training the model above took approx 1 hour on a T4 GPU (Google Colab).

In [2]:
# Let's generate a bit more protein sequence data using this model:
print(decode(m.generate(context, max_new_tokens=5000)[0].tolist()))


SYNSFMHTVLGQVSYLAYWEGDLSKGTIDCLYRCAHNYGVQRFIVYEKCGPAPALPRRNW
AFGTLVWLLCLVCLWAGGIPPTFAALSMQRALYYMMVSARSSGPIAQPTMFSSTSASPTL
LATLGSAVAKQVSTGYGTGFPRPYCISILGLIYVEICHAMFWNGLPYPLKGFEERSPRLK
YALLKSRRASKLKFSLIVVVTVLVSVRKSRNKDLEEGAQELLDLIQIAIVTHYGVGILFT
VPHARESIKDKIGGSIKISQMVAIYISIDEASAIAIPIKFSLILKIHCGSQWWDLLLIST
TVTAVQVVILYSNASMLLGLALSFFRLQILTGLNFPWIIHIGNFMTAIATNALSLGMSSF
FDGIIVDPVNDAARMISPLQVNDRVYLNSRRLDFNSIGISMSTALYLISVIICYPIESKK
NFKPKDMTTKDSLRTLWIFSAYSIIT
>
MDEKDIATIEDKWMAGEECVLLVEEEKNRIKTKVAEYNACNIGNSAFFYNSSGGTSLVIK
KKRRAKFPTLAAQISKRFKYYNLSINLREKLKLLRAAIKGCPSSIPWVILMLNRWKFPRK
KMSLLENPPKKTPFSSVLEADVFVYNPISLAIQGKTLIVKDGNVNASQHIYDQTTAFELD
RSTCCLSNFQFLDTATGSYHSYGTLSDDKVFFRQHAKYKYSEDELFPDRSFDQF
>
MEVLFRIPKYPLPDGETKYTLLVSKEKYGPVYLGSACNTQGLLVPNPPNPLVLASKYTTI
IAAGLIGNPTATPSYLSEERNIPTATMFGEIHVAGSILELYPYAPFSPAGKLDIIKSEFP
FVLKERVEILDNYYLRRADVKLRTRFIFGSSAVFKCIHKKELHSDDEQYINMCSKNQNVG
TSSITNAKFRQVVFDKEGKNCSPFCEGVLNGLHVLVDTPKSYFLDGDKVAYVRKGREIIK
VKKNLDTILVQPPGIKREPSSKSFSGLEMIGTEEAMVHFGLGGIGFLDTVGVHRKQTPN

**Do the generated sequences look like the input sequences?**

Yes.

**Do the generated sequences contain any secondary structure?**

Yes, Quick2D analysis (server accessible here: https://toolkit.tuebingen.mpg.de/tools/quick2d) shows predicted
secondary structure (alpha-helices, beta-sheets) for the generated sequences.

**Do the generated sequences contain any tertiary structure?**

Hmmm... not sure. Structure prediction using AlphaFold2
(accessible here: https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb) gives very low confidence (<50 pLDDT) structure predictions...
These are bad scores, and suggest that tertiary structure is
lacking from these generated sequences.

**Final:**

Still, for a simple Causal Language Model (i.e. it is trained *only*
on predicting the next token in a sequence) it does pretty well.

**NEXT:**

Try using the same data set with a Masked Language Model and
compare the results. (e.g. with https://github.com/lucidrains/mlm-pytorch/blob/master/mlm_pytorch/mlm_pytorch.py)